### Importa as bibliotecas a serem usadas

In [161]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import math
from os import mkdir, path

### Define caminho onde os dados .VEC estao salvos e cria uma pasta Dados_csv, onde amarzenara os dados gerados pelo script

In [138]:
caminho = Path('dados/tidi/')
arqs = sorted([str(i) for i in caminho.glob('**/*.VEC')])
arquivos_csv = Path('dados/tidi/dados_csv')
arquivos_csv.mkdir(exist_ok=True)

In [162]:
if not path.exists('dados/tidi/dados_zonal'):
    mkdir('dados/tidi/dados_zonal')
    mkdir('dados/tidi/dados_meridional')

### Loop para cada arquivo .VEC
### df.to_csv salva os dados

In [139]:
for arquivo in arqs:
    str_arq = str(arquivo)
    if 'D010_R01' not in str_arq:
        continue

    # comando para ler arquivo no formato netcdf
    df=xr.open_dataset(arquivo, engine='scipy')
    df= df.to_dataframe()
    #selecionando variaveis
    df2=df[['ascending','alt_retrieved','ut_date','ut_time','lat','lon','data_ok','u','var_u','v','var_v']]
    #selecionando latitude de São João do Cariri
    df2=df2.loc[df['lat'] >= -17]
    df2=df2.loc[df['lat'] <= 3]
    #selecionando longitude de São João do Cariri
    df2=df2.loc[df['lon'] >= 314]
    df2=df2.loc[df['lon'] <= 334]
    #Selecionando o intervalo de altitude
    df2=df2.loc[df['alt_retrieved'] >= 70]
    df2=df2.loc[df['alt_retrieved'] <= 100]
    df.ut_time
    time2=df.ut_time
    #convertendo o tempo para unidade de hora
    time2=time2/1e3/6e1/6e1
    df2['UT_time']=time2

    df2=df2[['ascending','alt_retrieved','ut_date','UT_time','lat','lon','data_ok','u','var_u','v','var_v']]
    df2
    # comando para salvar os arquivos
    #your_name= nome para salvar os arquivos
    df2.to_csv(f'dados/tidi/dados_csv/{str(arquivo)[16:-4]}.csv', sep=';')

### Converter dados csv para txt com ventos zonal e meridional

<strong> u = Zonal and v = Meridional

<strong> arquivo final: zonal/meridional_yyyymmdd <br>
<strong> colunas do arquivo final: tempo_corrido, 70km ... 100km, std_70km ... std_100km 

In [159]:
dict_months = {
    '01': [1, 31],
    '02': [32, 59],
    '03': [60, 90],
    '04': [91, 120],
    '05': [121, 151],
    '06': [152, 181],
    '07': [182, 212],
    '08': [213, 243],
    '09': [244, 273],
    '10': [274, 304],
    '11': [305, 334],
    '12': [335, 365],
}

dict_day_to_month = { i:k  for k,v in zip(dict_months.keys(), dict_months.values()) for i in range(v[0], v[1]+1)}

In [165]:
dict_day_to_month[1]

'01'

In [144]:
caminho = Path('dados/tidi/dados_csv/')
arquivos = sorted([i for i in caminho.glob('*.csv')])

In [154]:
def define_listas_df(df):
    lista_tempos = list(df.UT_time.value_counts().index)
    lista_alt = list(df.alt_retrieved.value_counts().index)
    lista_alt_df = [str(i)+'km' for i in lista_alt]
    lista_alt_std_df = ['std_'+str(i)+'km' for i in lista_alt]

    return lista_tempos, lista_alt_df, lista_alt_std_df

In [176]:
def retorna_df_convertido(df, lista_tempos, lista_alt_df, lista_alt_std_df, dir_vento):
    try:
        print(len(lista_tempos))
        df_converted = pd.DataFrame(columns=['tempo_decorrido']+lista_alt_df+lista_alt_std_df, index=range(0, len(lista_tempos)))

        for tempo,i in zip(lista_tempos, range(len(lista_tempos))):
            df_ = df[df.UT_time==tempo][['UT_time', 'alt_retrieved', dir_vento, 'var_'+dir_vento]].T
            df_converted.iloc[i]['tempo_decorrido'] = df_[i]['UT_time']
            for col, alt_df, alt_std_df in zip(df_.columns, lista_alt_df, lista_alt_std_df):
                df_converted.iloc[i][alt_df] = df_[col][dir_vento]
                df_converted.iloc[i][alt_std_df] = math.sqrt(df_[col]['var_'+dir_vento])
        
        return df_converted
    except:
        return pd.DataFrame()

In [178]:
for arq in arquivos:
    df = pd.read_csv(arq, sep=';')
    
    ano = str(arq)[29:33]
    dia = str(arq)[33:36]
    mes = dict_day_to_month[int(dia)]

    lista_tempos, lista_alt_df, lista_alt_std_df = define_listas_df(df)

    for dir_vento, nome_vento in zip(['u', 'v'], ['zonal', 'meridional']):
        df_convertido = retorna_df_convertido(df, lista_tempos, lista_alt_df, lista_alt_std_df, dir_vento)
        print(nome_vento+'_'+ano+mes+'_'+dia)
        if df_convertido.empty:
            continue
        else:
            df_convertido.to_csv(f'dados/tidi/dados_{nome_vento}/dados_{nome_vento}_{ano}{mes}_{dia}.txt', ';', index=False)

8
zonal_200501_001
8
meridional_200501_001
10
zonal_200501_002
10
meridional_200501_002
8
zonal_200501_003
8
meridional_200501_003
8
zonal_200501_004
8
meridional_200501_004
9
zonal_200501_005
9
meridional_200501_005
8
zonal_200501_006
8
meridional_200501_006
7
zonal_200501_007
7
meridional_200501_007
8
zonal_200501_008
8
meridional_200501_008
10
zonal_200501_009
10
meridional_200501_009
9
zonal_200501_010
9
meridional_200501_010
8
zonal_200501_011
8
meridional_200501_011
9
zonal_200501_012
9
meridional_200501_012
8
zonal_200501_013
8
meridional_200501_013
6
zonal_200501_014
6
meridional_200501_014
8
zonal_200501_015
8
meridional_200501_015
9
zonal_200501_016
9
meridional_200501_016
8
zonal_200501_017
8
meridional_200501_017
9
zonal_200501_018
9
meridional_200501_018
10
zonal_200501_019
10
meridional_200501_019
10
zonal_200501_020
10
meridional_200501_020
7
zonal_200501_021
7
meridional_200501_021
8
zonal_200501_022
8
meridional_200501_022
8
zonal_200501_023
8
meridional_200501_023
9
z